In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
import time
from sqlalchemy import inspect, create_engine
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.58M/6.58M [00:00<00:00, 15.2MB/s]


In [3]:
# HOSTNAME = "localhost"
# PORT = 5432
# DATABASE = "diapers_db"
# USERNAME = "postgres"
# PASSWORD = "password"
# path = f'postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOSTNAME}:{PORT}/{DATABASE}'
# engine = create_engine(path)

In [4]:
# conn = engine.connect()

In [3]:
#Website searching Data Analyst positions
indeed = "https://www.indeed.com/jobs?q=data+analyst"

browser.visit(indeed)



In [4]:
html = browser.html
html[0:500]

'<html dir="ltr" lang="en" class="js-focus-visible" data-js-focus-visible=""><head>\n    <link rel="shortcut icon" href="/images/favicon.ico">\n    <title>Data Analyst Jobs, Employment | Indeed.com</title>\n    <meta http-equiv="content-type" content="text/html; charset=utf-8">\n    <meta name="description" content="28,086 Data Analyst jobs available on Indeed.com. Apply to Data Analyst, Entry Level Data Analyst, Senior Data Analyst and more!">\n    <meta name="keywords" content="Data Analyst Jobs, Em'

In [5]:
soup = BeautifulSoup(browser.html, "html.parser")
#print(soup.prettify())

In [6]:
soup= BeautifulSoup(browser.html, 'lxml')
type(soup)

bs4.BeautifulSoup

In [7]:
#Find code to loop through web page and pull the title of the job and possibly the location, then find code to click the job and acquire the salary.
jobs = soup.find_all('div', {'class': 'cardOutline'})
len(jobs)


15

In [8]:
#Find the jobs titles for positions and the link IF available
job = jobs[0]
title = job.find('a', {"class": 'jcs-JobTitle'}).text
link = "www.indeed.com" + job.find('a', {"class": 'jcs-JobTitle'})["href"]


In [9]:
#Find company name and the link IF available
company = job.find('a', {"class": 'companyOverviewLink'}).text
company_link = "www.indeed.com" + job.find('a', {"class": 'companyOverviewLink'})["href"]

In [10]:
#Find the rating for the company/job
rating = job.find('span', {"class": 'ratingNumber'}).text
rating

'2.6'

In [11]:
#Find the location of the job listing
loc = job.find('div', {"class": 'companyLocation'}).text
loc

'Dallas, TX 75229\xa0(Northwest Dallas area)'

In [55]:
#Find the salary for the job listing
salary = job.find('div', {"class": 'salary-snippet-container'}).text
salary

'$83,000 - $88,000 a year'

In [14]:
#Pull the text only from the parent for job salary
job.find('svg', {"aria-label": "Job type"}).parent.text

'Full-time'

In [56]:
#
rows = []
browser.visit(indeed)
for i in range(1, 50):
    
    html = browser.html
    soup = BeautifulSoup(browser.html, "html.parser")
    jobs = soup.find_all('div', {'class': 'cardOutline'})
    
    
    for job in jobs:
        title = job.find('a', {"class": 'jcs-JobTitle'}).text
        link = "www.indeed.com" + job.find('a', {"class": 'jcs-JobTitle'})["href"]

        company_name = job.find('span', {'class': 'companyName'})
        if company_name: 
            company = company_name.text
        else:
            company = None
        
        company_con = job.find('a', {"class": 'companyOverviewLink'})
        if company_con: 
            company_link = "www.indeed.com" + company_con["href"]
        else:
            company_link = None

        rating_con = job.find('span', {"class": 'ratingNumber'})
        if rating_con: 
            rating = rating_con.text
        else:
            rating = None


        loc = job.find('div', {"class": 'companyLocation'}).text

        salary_con = job.find('div', {"class": 'salary-snippet-container'})
        if salary_con: 
            salary = salary_con.text
        else:
            salary = None


        status_con = job.find('svg', {"aria-label": "Job type"})
        if status_con:
            status = status_con.parent.text
        else:
            status = None

        data = {}
        data["job_title"] = title
        data["job_link"] = link
        data["company"] = company
        data["company_link"] = company_link
        data["company_rating"] = rating
        data["location"] = loc
        data["salary"] = salary
        data["status"] = status
        rows.append(data)
    
    browser.links.find_by_text(f"{i+1}").click()
        
df = pd.DataFrame(rows)
df.head()

,job_title,job_link,company,company_link,company_rating,location,salary,status
0,ISD Architect I (Senior Data Analyst),www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0C1...,Navy Federal Credit Union,www.indeed.com/cmp/Navy-Federal-Credit-Union,3.9,"Pensacola, FL","$95,600 - $163,500 a year",Full-time +1
1,Business Data Analyst - Operations,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Av...,Summit Companies,www.indeed.com/cmp/Summit-Companies,2.9,"Mendota Heights, MN 55120","$65,000 - $85,000 a year",Full-time
2,Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bb...,Allied Stone Inc,"www.indeed.com/cmp/Allied-Stone,-Inc.",2.6,"Dallas, TX 75229 (Northwest Dallas area)",None,Full-time
3,Entry level Business /Data Analyst,www.indeed.com/company/TRESUME/jobs/Entry-Leve...,TRESUME,None,None,"Dallas, TX",$30 - $35 an hour,Full-time
4,Data Analyst,www.indeed.com/rc/clk?jk=45fb1e57894adc6d&fcci...,Disney,www.indeed.com/cmp/The-Walt-Disney-Company,4.1,"Remote in Santa Monica, CA 90401","$87,900 - $137,300 a year",None


In [57]:
len(df)

735

In [58]:
df["company_rating"] = pd.to_numeric(df.company_rating)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   job_title       735 non-null    object 
 1   job_link        735 non-null    object 
 2   company         733 non-null    object 
 3   company_link    571 non-null    object 
 4   company_rating  571 non-null    float64
 5   location        735 non-null    object 
 6   salary          338 non-null    object 
 7   status          551 non-null    object 
dtypes: float64(1), object(7)
memory usage: 46.1+ KB


In [59]:
df.status.value_counts()

Full-time       440
Contract         49
Full-time +1     44
Full-time +2      6
Part-time         5
Contract +1       3
Temporary         2
Part-time +1      1
Temp-to-hire      1
Name: status, dtype: int64

In [60]:
#combine with regex s=space d=catch or match all numbers \+ = plus sign matches plus sign, regex meaning.
df["status"] = df.status.str.replace("\s\+\d","")

C:\Users\19722\AppData\Local\Temp\ipykernel_7444\152645899.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["status"] = df.status.str.replace("\s\+\d","")


In [61]:
df.status.value_counts()

Full-time       490
Contract         52
Part-time         6
Temporary         2
Temp-to-hire      1
Name: status, dtype: int64

In [62]:
#Status column -  ID, name, last_updated
df.status.unique()

array(['Full-time', None, 'Contract', 'Part-time', 'Temporary',
       'Temp-to-hire'], dtype=object)

In [63]:
#Status lookup table
status = df.status.dropna().unique()
status_df = pd.DataFrame()
status_df["status"] = status
status_df

,status
0,Full-time
1,Contract
2,Part-time
3,Temporary
4,Temp-to-hire


In [85]:
status_df.to_csv('status.csv')

In [64]:
#Company lookup table, revised to drop the 2nd Insight Global below
companies = df.loc[:,["company","company_link", "company_rating"]].drop_duplicates().reset_index(drop=True)
companies.company.value_counts()

Navy Federal Credit Union        1
Dept of State Health Services    1
RD DIGITAL SOLUTIONS             1
Corsearch                        1
LeewayHertz                      1
                                ..
Colgate-Palmolive                1
crox                             1
HP                               1
MNTN                             1
Scalepex                         1
Name: company, Length: 627, dtype: int64

In [65]:
companies.loc[companies.company == "Insight Global"]

,company,company_link,company_rating
114,Insight Global,None,NaN


In [66]:
df.loc[df.company == "Insight Global", "company_link"] = "www.indeed.com/cmp/Insight-Global"
df.loc[df.company == "Insight Global", "company_rating"] = 3.9

In [67]:
companies = df.loc[:,["company","company_link", "company_rating"]].drop_duplicates().reset_index(drop=True)
companies.company.value_counts()

Navy Federal Credit Union        1
Dept of State Health Services    1
RD DIGITAL SOLUTIONS             1
Corsearch                        1
LeewayHertz                      1
                                ..
Colgate-Palmolive                1
crox                             1
HP                               1
MNTN                             1
Scalepex                         1
Name: company, Length: 627, dtype: int64

In [84]:
#Companies lookup table
companies

,company,company_link,company_rating
0,Navy Federal Credit Union,www.indeed.com/cmp/Navy-Federal-Credit-Union,3.9
1,Summit Companies,www.indeed.com/cmp/Summit-Companies,2.9
2,Allied Stone Inc,"www.indeed.com/cmp/Allied-Stone,-Inc.",2.6
3,TRESUME,None,NaN
4,Disney,www.indeed.com/cmp/The-Walt-Disney-Company,4.1
...,...,...,...
623,IQVIA,www.indeed.com/cmp/Iqvia,3.8
624,Change Healthcare,www.indeed.com/cmp/Change-Healthcare,3.0
625,Citi,www.indeed.com/cmp/Citi,3.9
626,Pratt & Whitney,www.indeed.com/cmp/Pratt-&-Whitney,3.9


In [68]:
#State 
def getState(loc):
    if "," in loc:
        state = loc.replace("\xa0", " ").split(",")[1].strip().split(" ")[0].strip()
    else:
        state = None
        
    return(state)

In [69]:
#Find the locations with"Downtown area"
getState("Austin, TX (Downtown area)")

'TX'

In [70]:
#
df["state"] = df.location.apply(lambda x: getState(x))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   job_title       735 non-null    object 
 1   job_link        735 non-null    object 
 2   company         733 non-null    object 
 3   company_link    572 non-null    object 
 4   company_rating  572 non-null    float64
 5   location        735 non-null    object 
 6   salary          338 non-null    object 
 7   status          551 non-null    object 
 8   state           527 non-null    object 
dtypes: float64(1), object(8)
memory usage: 51.8+ KB


In [71]:
#
df.state.value_counts()

TX    223
CA     54
NY     47
VA     18
IL     17
GA     16
FL     10
WA     10
PA      9
MN      9
CO      9
MD      8
NC      8
MA      8
OK      7
AZ      7
MI      6
DC      5
NJ      5
OH      5
UT      5
TN      4
WI      4
MS      4
CT      4
OR      3
SC      3
NM      3
IA      2
MO      2
KY      2
IN      2
KS      1
RI      1
AR      1
NV      1
LA      1
DE      1
VT      1
ME      1
Name: state, dtype: int64

In [72]:
#
df.loc[df.location.str.contains("\(Downtown")]

,job_title,job_link,company,company_link,company_rating,location,salary,status,state
104,"Cloud Consultant, Big Data Analytics, Looker, ...",www.indeed.com/rc/clk?jk=abf4ea6cd747a933&fcci...,Google,www.indeed.com/cmp/Google,4.3,"Austin, TX (Downtown area)",None,Full-time,TX
148,Data Analyst,www.indeed.com/company/Imprimis/jobs/Data-Anal...,Imprimis,www.indeed.com/cmp/Imprimis,4.4,"Dallas, TX 75201 (Downtown area)",$26 - $33 an hour,Contract,TX
176,Part-Time Data Entry Analyst - Solvas Financia...,www.indeed.com/rc/clk?jk=3f46b3d784904a0a&fcci...,Deloitte,www.indeed.com/cmp/Deloitte,3.9,"Dallas, TX 75201 (Downtown area)",$20 an hour,Full-time,TX
179,Senior Analyst Data & Analytics Visualizations...,www.indeed.com/rc/clk?jk=618489124496ca55&fcci...,The Coca-Cola Company,www.indeed.com/cmp/The-Coca--cola-Company,3.8,"Atlanta, GA 30313 (Downtown area)",None,Full-time,GA
205,Jr. Data Analyst/ Assistant,www.indeed.com/company/RPO-International/jobs/...,RPO International,None,NaN,"Houston, TX 77002 (Downtown area)",Up to $28 an hour,Contract,TX
292,Data Analytics and Business Intelligence | Ana...,www.indeed.com/rc/clk?jk=381b673be9ee0b6c&fcci...,Goldman Sachs,www.indeed.com/cmp/Goldman-Sachs,4.0,"Salt Lake City, UT 84111 (Downtown area)",None,None,UT
293,"THECB - Data Analyst V, Visualization",www.indeed.com/rc/clk?jk=87d4703c7888a6f7&fcci...,TEXAS HIGHER EDUCATION COORDINATING BOARD,www.indeed.com/cmp/Texas-Higher-Education-Coor...,3.8,"Austin, TX 78701 (Downtown area)","$6,017 - $8,266 a month",Full-time,TX
386,Policy Data Analyst (Research Specialist I) 20...,www.indeed.com/company/Governor-of-Texas/jobs/...,Office of the Governor,None,NaN,"Austin, TX 78701 (Downtown area)","$4,000 - $4,333 a month",Full-time,TX
416,Data Analyst - Student Health,www.indeed.com/rc/clk?jk=666c016ed596d204&fcci...,Keck Medical Center of USC,www.indeed.com/cmp/Keck-Medicine-of-Usc,3.8,"Los Angeles, CA 90015 (Downtown area)",None,None,CA
509,Data Analyst,www.indeed.com/rc/clk?jk=d65bcc5382483a02&fcci...,Baylor University,www.indeed.com/cmp/Baylor-University,4.4,"Waco, TX 76706 (Downtown area)",None,Full-time,TX


In [73]:
df.iloc[71].location

'Detroit, MI'

In [74]:
#State look-up table
state = df.state.dropna().unique()
state_df = pd.DataFrame()
state_df["state"] = state
state_df

,state
0,FL
1,MN
2,TX
3,CA
4,OR
5,IL
6,GA
7,VA
8,NC
9,NY


In [75]:
#
def getRemote(loc):
    if "remote" in loc.lower():
        return(True)
    else:
        return(False)
        
  

In [77]:
#
df["is_remote"] = df.location.apply(lambda x: getRemote(x))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   job_title       735 non-null    object 
 1   job_link        735 non-null    object 
 2   company         733 non-null    object 
 3   company_link    572 non-null    object 
 4   company_rating  572 non-null    float64
 5   location        735 non-null    object 
 6   salary          338 non-null    object 
 7   status          551 non-null    object 
 8   state           527 non-null    object 
 9   is_remote       735 non-null    bool   
dtypes: bool(1), float64(1), object(8)
memory usage: 52.5+ KB


In [78]:
df.is_remote.value_counts()

True     368
False    367
Name: is_remote, dtype: int64

In [101]:
df.is_remote.info()

<class 'pandas.core.series.Series'>
RangeIndex: 735 entries, 0 to 734
Series name: is_remote
Non-Null Count  Dtype
--------------  -----
735 non-null    bool 
dtypes: bool(1)
memory usage: 863.0 bytes


In [102]:
is_remote = df.is_remote.dropna().unique()
is_remote_df = pd.DataFrame()
is_remote_df["is_remote"] = is_remote
is_remote_df

,is_remote
0,False
1,True


In [103]:
df.head()

,job_title,job_link,company,company_link,company_rating,location,salary,status,state,is_remote,is_ad
0,ISD Architect I (Senior Data Analyst),www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0C1...,Navy Federal Credit Union,www.indeed.com/cmp/Navy-Federal-Credit-Union,3.9,"Pensacola, FL","$95,600 - $163,500 a year",Full-time,FL,False,True
1,Business Data Analyst - Operations,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Av...,Summit Companies,www.indeed.com/cmp/Summit-Companies,2.9,"Mendota Heights, MN 55120","$65,000 - $85,000 a year",Full-time,MN,False,True
2,Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bb...,Allied Stone Inc,"www.indeed.com/cmp/Allied-Stone,-Inc.",2.6,"Dallas, TX 75229 (Northwest Dallas area)",None,Full-time,TX,False,True
3,Entry level Business /Data Analyst,www.indeed.com/company/TRESUME/jobs/Entry-Leve...,TRESUME,None,NaN,"Dallas, TX",$30 - $35 an hour,Full-time,TX,False,False
4,Data Analyst,www.indeed.com/rc/clk?jk=45fb1e57894adc6d&fcci...,Disney,www.indeed.com/cmp/The-Walt-Disney-Company,4.1,"Remote in Santa Monica, CA 90401","$87,900 - $137,300 a year",None,CA,True,False


In [79]:
#Find the companies paying for ads
def getAd(link):
    if "pagead" in link.lower():
        return(True)
    else:
        return(False)

In [80]:
df["is_ad"] = df.job_link.apply(lambda x: getAd(x))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   job_title       735 non-null    object 
 1   job_link        735 non-null    object 
 2   company         733 non-null    object 
 3   company_link    572 non-null    object 
 4   company_rating  572 non-null    float64
 5   location        735 non-null    object 
 6   salary          338 non-null    object 
 7   status          551 non-null    object 
 8   state           527 non-null    object 
 9   is_remote       735 non-null    bool   
 10  is_ad           735 non-null    bool   
dtypes: bool(2), float64(1), object(8)
memory usage: 53.2+ KB


In [81]:
df.is_ad.value_counts()

False    620
True     115
Name: is_ad, dtype: int64

In [82]:
#Paying for Ads, add to main job table as bool
df.groupby("company").is_ad.sum().sort_values(ascending=False)

company
Kforce                                 4
Guidehouse                             2
DISYS                                  2
Vanguard                               2
Cancer Treatment Centers of America    2
                                      ..
GENUENT                                0
Frontiers                              0
Front End                              0
FreedomCare                            0
sgsco                                  0
Name: is_ad, Length: 627, dtype: int64

In [95]:
#Create a CSV for companies 
companies.to_csv('companies.csv', index=False)

In [96]:
companies_file = pd.read_csv('companies.csv')
companies_file

,company,company_link,company_rating
0,Navy Federal Credit Union,www.indeed.com/cmp/Navy-Federal-Credit-Union,3.9
1,Summit Companies,www.indeed.com/cmp/Summit-Companies,2.9
2,Allied Stone Inc,"www.indeed.com/cmp/Allied-Stone,-Inc.",2.6
3,TRESUME,NaN,NaN
4,Disney,www.indeed.com/cmp/The-Walt-Disney-Company,4.1
...,...,...,...
623,IQVIA,www.indeed.com/cmp/Iqvia,3.8
624,Change Healthcare,www.indeed.com/cmp/Change-Healthcare,3.0
625,Citi,www.indeed.com/cmp/Citi,3.9
626,Pratt & Whitney,www.indeed.com/cmp/Pratt-&-Whitney,3.9


In [97]:
#Create a CSV for States
state_df.to_csv('state.csv', index=False)

In [99]:
state_file = pd.read_csv('state.csv')
state_file

,state
0,FL
1,MN
2,TX
3,CA
4,OR
5,IL
6,GA
7,VA
8,NC
9,NY


In [93]:
#Create a CSV for status (Full-time/part-time)
status_df.to_csv('status.csv', index=False)

In [94]:
status_file = pd.read_csv('status.csv')
status_file

,status
0,Full-time
1,Contract
2,Part-time
3,Temporary
4,Temp-to-hire


In [83]:
#Don't worry about it
# def getCity(loc):
#     if "," in loc:
#         city_list = loc.replace("\xa0", " ").split(",")[0].strip().split(" ")
#         city = city_list[0] if len(city_list) ==1 else city_list[1]
#     else:
#         city = None
        
#     return(city)

In [ ]:
# df["city"] = df.location.apply(lambda x: getCity(x))
# df.info()

In [ ]:
# #
# df.city.value_counts()

In [ ]:
#
# df.loc[df.location.str.contains("in")]

In [ ]:
#Set-up sleep function if needed
# browser.visit(dice_lyst)
# time.sleep(2)

In [104]:
df.head()

,job_title,job_link,company,company_link,company_rating,location,salary,status,state,is_remote,is_ad
0,ISD Architect I (Senior Data Analyst),www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0C1...,Navy Federal Credit Union,www.indeed.com/cmp/Navy-Federal-Credit-Union,3.9,"Pensacola, FL","$95,600 - $163,500 a year",Full-time,FL,False,True
1,Business Data Analyst - Operations,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Av...,Summit Companies,www.indeed.com/cmp/Summit-Companies,2.9,"Mendota Heights, MN 55120","$65,000 - $85,000 a year",Full-time,MN,False,True
2,Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bb...,Allied Stone Inc,"www.indeed.com/cmp/Allied-Stone,-Inc.",2.6,"Dallas, TX 75229 (Northwest Dallas area)",None,Full-time,TX,False,True
3,Entry level Business /Data Analyst,www.indeed.com/company/TRESUME/jobs/Entry-Leve...,TRESUME,None,NaN,"Dallas, TX",$30 - $35 an hour,Full-time,TX,False,False
4,Data Analyst,www.indeed.com/rc/clk?jk=45fb1e57894adc6d&fcci...,Disney,www.indeed.com/cmp/The-Walt-Disney-Company,4.1,"Remote in Santa Monica, CA 90401","$87,900 - $137,300 a year",None,CA,True,False


In [105]:
status_df.head()

,status
0,Full-time
1,Contract
2,Part-time
3,Temporary
4,Temp-to-hire


In [106]:
import pandas as pd
from sqlalchemy import create_engine, inspect

In [115]:
USERNAME = "postgres"
PASSWORD = "password"
URL = "localhost"
PORT = 5432
DATABASE = "Jobs_db"

connection_str = f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{URL}:{PORT}/{DATABASE}"
#print(connection_str)

engine = create_engine(connection_str)

In [122]:
# create the inspector and connect to the engine
inspector_gadget = inspect(engine)

tables = inspector_gadget.get_table_names()
for table in tables:
    print(table)
    
    # get all columns in table
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column)
    print()

company
{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': "nextval('company_id_seq'::regclass)", 'autoincrement': True, 'comment': None}
{'name': 'company', 'type': VARCHAR(length=100), 'nullable': False, 'default': None, 'autoincrement': False, 'comment': None}
{'name': 'company_link', 'type': VARCHAR(length=500), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}
{'name': 'company_rating', 'type': DOUBLE_PRECISION(precision=53), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}
{'name': 'last_updated', 'type': TIMESTAMP(), 'nullable': False, 'default': 'LOCALTIMESTAMP', 'autoincrement': False, 'comment': None}

job
{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': "nextval('job_id_seq'::regclass)", 'autoincrement': True, 'comment': None}
{'name': 'job_title', 'type': VARCHAR(length=100), 'nullable': False, 'default': None, 'autoincrement': False, 'comment': None}
{'name': 'job_link', 'type': VARCHAR(length

In [123]:
status_df.head()

,status
0,Full-time
1,Contract
2,Part-time
3,Temporary
4,Temp-to-hire


In [124]:
status_df.to_sql("status", engine, method="multi", if_exists = "append", index=False)

5

In [125]:
state_df.to_sql("state", engine, method="multi", if_exists = "append", index=False)

40

In [126]:
companies.to_sql("company", engine, method="multi", if_exists = "append", index=False)

IntegrityError: (psycopg2.errors.NotNullViolation) null value in column "company" of relation "company" violates not-null constraint
DETAIL:  Failing row contains (238, null, null, null, 2022-12-20 20:04:48.799364).

[SQL: INSERT INTO company (company, company_link, company_rating) VALUES (%(company_m0)s, %(company_link_m0)s, %(company_rating_m0)s), (%(company_m1)s, %(company_link_m1)s, %(company_rating_m1)s), (%(company_m2)s, %(company_link_m2)s, %(company_rating_m2)s), (%(company_m3)s, %(company_link_m3)s, %(company_rating_m3)s), (%(company_m4)s, %(company_link_m4)s, %(company_rating_m4)s), (%(company_m5)s, %(company_link_m5)s, %(company_rating_m5)s), (%(company_m6)s, %(company_link_m6)s, %(company_rating_m6)s), (%(company_m7)s, %(company_link_m7)s, %(company_rating_m7)s), (%(company_m8)s, %(company_link_m8)s, %(company_rating_m8)s), (%(company_m9)s, %(company_link_m9)s, %(company_rating_m9)s), (%(company_m10)s, %(company_link_m10)s, %(company_rating_m10)s), (%(company_m11)s, %(compa

In [127]:
companies.head()

,company,company_link,company_rating
0,Navy Federal Credit Union,www.indeed.com/cmp/Navy-Federal-Credit-Union,3.9
1,Summit Companies,www.indeed.com/cmp/Summit-Companies,2.9
2,Allied Stone Inc,"www.indeed.com/cmp/Allied-Stone,-Inc.",2.6
3,TRESUME,None,NaN
4,Disney,www.indeed.com/cmp/The-Walt-Disney-Company,4.1


In [128]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 628 entries, 0 to 627
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   company         627 non-null    object 
 1   company_link    474 non-null    object 
 2   company_rating  474 non-null    float64
dtypes: float64(1), object(2)
memory usage: 14.8+ KB


In [129]:
companies.loc[pd.isnull(companies.company)]

,company,company_link,company_rating
237,None,None,NaN


In [131]:
#Drop one company with no name, inclue in writeup
companies = companies.loc[pd.notnull(companies.company)].reset_index(drop=True)
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   company         627 non-null    object 
 1   company_link    474 non-null    object 
 2   company_rating  474 non-null    float64
dtypes: float64(1), object(2)
memory usage: 14.8+ KB


In [133]:
companies.to_sql("company", engine, method="multi", if_exists = "append", index=False)

627

In [134]:
df.head()

,job_title,job_link,company,company_link,company_rating,location,salary,status,state,is_remote,is_ad
0,ISD Architect I (Senior Data Analyst),www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0C1...,Navy Federal Credit Union,www.indeed.com/cmp/Navy-Federal-Credit-Union,3.9,"Pensacola, FL","$95,600 - $163,500 a year",Full-time,FL,False,True
1,Business Data Analyst - Operations,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Av...,Summit Companies,www.indeed.com/cmp/Summit-Companies,2.9,"Mendota Heights, MN 55120","$65,000 - $85,000 a year",Full-time,MN,False,True
2,Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bb...,Allied Stone Inc,"www.indeed.com/cmp/Allied-Stone,-Inc.",2.6,"Dallas, TX 75229 (Northwest Dallas area)",None,Full-time,TX,False,True
3,Entry level Business /Data Analyst,www.indeed.com/company/TRESUME/jobs/Entry-Leve...,TRESUME,None,NaN,"Dallas, TX",$30 - $35 an hour,Full-time,TX,False,False
4,Data Analyst,www.indeed.com/rc/clk?jk=45fb1e57894adc6d&fcci...,Disney,www.indeed.com/cmp/The-Walt-Disney-Company,4.1,"Remote in Santa Monica, CA 90401","$87,900 - $137,300 a year",None,CA,True,False


In [137]:
status2 = pd.read_sql("select id as status_id, status from status", engine)
status2.head()

,status_id,status
0,1,Full-time
1,2,Contract
2,3,Part-time
3,4,Temporary
4,5,Temp-to-hire


In [139]:
df = pd.merge(df, status2, on = "status", how ="left")
df.drop("status", axis=1, inplace=True)
df.head()

,job_title,job_link,company,company_link,company_rating,location,salary,state,is_remote,is_ad,status_id
0,ISD Architect I (Senior Data Analyst),www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0C1...,Navy Federal Credit Union,www.indeed.com/cmp/Navy-Federal-Credit-Union,3.9,"Pensacola, FL","$95,600 - $163,500 a year",FL,False,True,1.0
1,Business Data Analyst - Operations,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Av...,Summit Companies,www.indeed.com/cmp/Summit-Companies,2.9,"Mendota Heights, MN 55120","$65,000 - $85,000 a year",MN,False,True,1.0
2,Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bb...,Allied Stone Inc,"www.indeed.com/cmp/Allied-Stone,-Inc.",2.6,"Dallas, TX 75229 (Northwest Dallas area)",None,TX,False,True,1.0
3,Entry level Business /Data Analyst,www.indeed.com/company/TRESUME/jobs/Entry-Leve...,TRESUME,None,NaN,"Dallas, TX",$30 - $35 an hour,TX,False,False,1.0
4,Data Analyst,www.indeed.com/rc/clk?jk=45fb1e57894adc6d&fcci...,Disney,www.indeed.com/cmp/The-Walt-Disney-Company,4.1,"Remote in Santa Monica, CA 90401","$87,900 - $137,300 a year",CA,True,False,NaN


In [140]:
state2 = pd.read_sql("select id as state_id, state from state", engine)
df = pd.merge(df, state2, on = "state", how ="left")
df.drop("state", axis=1, inplace=True)
df.head()

,job_title,job_link,company,company_link,company_rating,location,salary,is_remote,is_ad,status_id,state_id
0,ISD Architect I (Senior Data Analyst),www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0C1...,Navy Federal Credit Union,www.indeed.com/cmp/Navy-Federal-Credit-Union,3.9,"Pensacola, FL","$95,600 - $163,500 a year",False,True,1.0,1.0
1,Business Data Analyst - Operations,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Av...,Summit Companies,www.indeed.com/cmp/Summit-Companies,2.9,"Mendota Heights, MN 55120","$65,000 - $85,000 a year",False,True,1.0,2.0
2,Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bb...,Allied Stone Inc,"www.indeed.com/cmp/Allied-Stone,-Inc.",2.6,"Dallas, TX 75229 (Northwest Dallas area)",None,False,True,1.0,3.0
3,Entry level Business /Data Analyst,www.indeed.com/company/TRESUME/jobs/Entry-Leve...,TRESUME,None,NaN,"Dallas, TX",$30 - $35 an hour,False,False,1.0,3.0
4,Data Analyst,www.indeed.com/rc/clk?jk=45fb1e57894adc6d&fcci...,Disney,www.indeed.com/cmp/The-Walt-Disney-Company,4.1,"Remote in Santa Monica, CA 90401","$87,900 - $137,300 a year",True,False,NaN,4.0


In [141]:
company2 = pd.read_sql("select id as company_id, company from company", engine)
df = pd.merge(df, company2, on = "company", how ="left")
df.drop(["company", "company_link", "company_rating"], axis=1, inplace=True)
df.head()

,job_title,job_link,location,salary,is_remote,is_ad,status_id,state_id,company_id
0,ISD Architect I (Senior Data Analyst),www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0C1...,"Pensacola, FL","$95,600 - $163,500 a year",False,True,1.0,1.0,1.0
1,Business Data Analyst - Operations,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Av...,"Mendota Heights, MN 55120","$65,000 - $85,000 a year",False,True,1.0,2.0,2.0
2,Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bb...,"Dallas, TX 75229 (Northwest Dallas area)",None,False,True,1.0,3.0,3.0
3,Entry level Business /Data Analyst,www.indeed.com/company/TRESUME/jobs/Entry-Leve...,"Dallas, TX",$30 - $35 an hour,False,False,1.0,3.0,4.0
4,Data Analyst,www.indeed.com/rc/clk?jk=45fb1e57894adc6d&fcci...,"Remote in Santa Monica, CA 90401","$87,900 - $137,300 a year",True,False,NaN,4.0,5.0


In [146]:
#Database
df.to_sql("job", engine, method="multi", if_exists = "append", index=False)

735

In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 735 entries, 0 to 734
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   job_title   735 non-null    object 
 1   job_link    735 non-null    object 
 2   location    735 non-null    object 
 3   salary      338 non-null    object 
 4   is_remote   735 non-null    bool   
 5   is_ad       735 non-null    bool   
 6   status_id   551 non-null    float64
 7   state_id    527 non-null    float64
 8   company_id  733 non-null    float64
dtypes: bool(2), float64(3), object(4)
memory usage: 47.4+ KB
